# NB1.5 — Streaming vs Batch (WebSockets & Polling)

En esta práctica implementamos **Streaming** con WebSockets y **Batch frecuente** con HTTP Polling. Guardaremos todo en **JSON Lines** en `../../data/` y compararemos latencia y continuidad.

**Objetivos**
- Entender diferencias clave **Batch vs Streaming**.
- Ejecutar un cliente **WebSocket** para ingesta continua.
- Ejecutar **HTTP Polling** como micro-batch.
- Comparar resultados en pandas.


## 1) Dependencias
Instalamos librerías requeridas para los scripts en `src/streaming/`.

In [ ]:
!pip install -q websockets requests pandas

## 2) Streaming (WebSocket)

**Salida:** `../../data/stream_ws_YYYY-MM-DD.jsonl`

In [9]:
# Limita duración/eventos para la clase
!WS_MAX_EVENTS=150 WS_MAX_SECONDS=120 python ../../src/streaming/stream_dual_ws.py

/Users/didiergamboa/Documents/Tecnológico de Software/Cuatrimestres/2025 Q3 Septiembre-Diciembre/Fundamentos de Big Data/Proyecto Integrador - Fundamentos de Ingeniería de Datos/fundamentos-ingenieria-datos/notebooks/tutoriales/../../src/streaming/stream_dual_ws.py:8: DeprecationWarning: websockets.exceptions.InvalidStatusCode is deprecated
  from websockets.exceptions import ConnectionClosed, InvalidStatusCode
[DONE] Eventos: 150
/Users/didiergamboa/Documents/Tecnológico de Software/Cuatrimestres/2025 Q3 Septiembre-Diciembre/Fundamentos de Big Data/Proyecto Integrador - Fundamentos de Ingeniería de Datos/fundamentos-ingenieria-datos/data/stream_ws_2025-09-24.jsonl


## 3) Polling (HTTP)

**Salida:** `../../data/poll_bitcoin_YYYY-MM-DD.jsonl`

In [10]:
!python ../../src/streaming/poll_coincap_http.py

[POLL] 1/20 -> {'ts': '2025-09-24T21:04:40.311738+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.39}
[POLL] 2/20 -> {'ts': '2025-09-24T21:04:45.582810+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.39}
[POLL] 3/20 -> {'ts': '2025-09-24T21:04:50.851553+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.39}
[POLL] 4/20 -> {'ts': '2025-09-24T21:04:56.124504+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.4}
[POLL] 5/20 -> {'ts': '2025-09-24T21:05:01.408466+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.39}
[POLL] 6/20 -> {'ts': '2025-09-24T21:05:06.697973+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.4}
[POLL] 7/20 -> {'ts': '2025-09-24T21:05:12.298638+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 113501.4}
[POLL] 8/20 -> {'ts': '2025-09-24T21:05:17.569889+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'pr

## 4) Cargar y comparar en pandas
Leemos ambos `.jsonl` y observamos diferencias de granularidad/latencia (aprox.).

In [ ]:
from pathlib import Path
import pandas as pd

data_dir = Path("../../data")

# STREAM
stream_file = sorted(data_dir.glob("stream_ws_*.jsonl"))[-1]
df_stream = pd.read_json(stream_file, lines=True)
print("STREAM:", stream_file)
display(df_stream.head())

# POLL 
poll_files = sorted(list(data_dir.glob("poll_binance_*.jsonl")) + list(data_dir.glob("poll_coincap_*.jsonl")))
poll_file = poll_files[-1]
df_poll = pd.read_json(poll_file, lines=True)
print("POLL:", poll_file)
display(df_poll.head())


STREAM: ../../data/stream_ws_2025-09-24.jsonl


,_probe,ts,source,instrument,price,currency,qty,trade_id
0,1.0,2025-09-24T20:31:35.921395+00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2025-09-24T20:31:38.079193+00:00,binance,BTCUSDT,113492.94,USDT,0.00041,5.254305e+09
2,NaN,2025-09-24T20:31:38.080583+00:00,binance,BTCUSDT,113492.94,USDT,0.00080,5.254305e+09
3,NaN,2025-09-24T20:31:38.081120+00:00,binance,BTCUSDT,113492.94,USDT,0.01344,5.254305e+09
4,NaN,2025-09-24T20:31:38.082035+00:00,binance,BTCUSDT,113492.94,USDT,0.00014,5.254305e+09


POLL: ../../data/poll_binance_BTCUSDT_2025-09-24.jsonl


,ts,source,instrument,price_usd
0,2025-09-24T20:38:10.342191+00:00,binance,BTCUSDT,113509.73
1,2025-09-24T20:38:15.605021+00:00,binance,BTCUSDT,113509.73
2,2025-09-24T20:38:20.866912+00:00,binance,BTCUSDT,113509.74
3,2025-09-24T20:38:26.129273+00:00,binance,BTCUSDT,113504.04
4,2025-09-24T20:38:31.390429+00:00,binance,BTCUSDT,113483.82


## 5) Reflexión (respuestas cortas)
**¿Quién tiene menor latencia?**  

**¿Qué pasa si el stream se cae?** 

**¿Cuál genera más duplicados o huecos temporales?**
